<a href="https://colab.research.google.com/github/YashFiles/threatLENS/blob/main/malware_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [80]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [114]:
!pip install pefile scikit-learn imbalanced-learn xgboost lightgbm -q


In [82]:
import pefile
import pandas as pd
import os

# Function to extract PE headers from an .exe file
def extract_pe_headers(file_path):
    pe = pefile.PE(file_path)
    pe_headers = {
        'size': pe.OPTIONAL_HEADER.SizeOfImage,
        'vsize': pe.OPTIONAL_HEADER.SizeOfHeaders,
        'has_debug': int(hasattr(pe, 'DIRECTORY_ENTRY_DEBUG')),
        'exports_counts': len(getattr(pe, 'DIRECTORY_ENTRY_EXPORT', [])),
        'imports_counts': len(getattr(pe, 'DIRECTORY_ENTRY_IMPORT', [])),
        'has_relocations': int(hasattr(pe, 'DIRECTORY_ENTRY_BASERELOC')),
        'has_resources': int(hasattr(pe, 'DIRECTORY_ENTRY_RESOURCE')),
        'has_signature': int(hasattr(pe, 'DIRECTORY_ENTRY_SECURITY')),
        'has_tls': int(hasattr(pe, 'DIRECTORY_ENTRY_TLS')),
        'symbols': pe.FILE_HEADER.NumberOfSymbols,
        'coff.timestamp': pe.FILE_HEADER.TimeDateStamp,
        'coff.machine': pe.FILE_HEADER.Machine,
        'coff.characteristics': pe.FILE_HEADER.Characteristics,
        'optional.subsystem': pe.OPTIONAL_HEADER.Subsystem,
        'optional.dll_characteristics': pe.OPTIONAL_HEADER.DllCharacteristics,
        'optional.magic': pe.OPTIONAL_HEADER.Magic,
        'optional.major_image_version': pe.OPTIONAL_HEADER.MajorImageVersion,
        'optional.minor_image_version': pe.OPTIONAL_HEADER.MinorImageVersion,
        'optional.major_linker_version': pe.OPTIONAL_HEADER.MajorLinkerVersion,
        'optional.minor_linker_version': pe.OPTIONAL_HEADER.MinorLinkerVersion,
        'optional.major_operating_system_version': pe.OPTIONAL_HEADER.MajorOperatingSystemVersion,
        'optional.minor_operating_system_version': pe.OPTIONAL_HEADER.MinorOperatingSystemVersion,
        'optional.major_subsystem_version': pe.OPTIONAL_HEADER.MajorSubsystemVersion,
        'optional.minor_subsystem_version': pe.OPTIONAL_HEADER.MinorSubsystemVersion,
        'optional.sizeof_code': pe.OPTIONAL_HEADER.SizeOfCode,
        'optional.sizeof_headers': pe.OPTIONAL_HEADER.SizeOfHeaders,
        'optional.sizeof_heap_commit': pe.OPTIONAL_HEADER.SizeOfHeapCommit,
    }
    return pe_headers

In [83]:
# Example usage
exe_file = "/content/drive/MyDrive/NFS13.exe"
pe_headers = extract_pe_headers(exe_file)
print(pe_headers)

{'size': 14389248, 'vsize': 1024, 'has_debug': 1, 'exports_counts': 0, 'imports_counts': 18, 'has_relocations': 0, 'has_resources': 1, 'has_signature': 0, 'has_tls': 1, 'symbols': 0, 'coff.timestamp': 1349449599, 'coff.machine': 332, 'coff.characteristics': 291, 'optional.subsystem': 2, 'optional.dll_characteristics': 33024, 'optional.magic': 267, 'optional.major_image_version': 0, 'optional.minor_image_version': 0, 'optional.major_linker_version': 10, 'optional.minor_linker_version': 0, 'optional.major_operating_system_version': 5, 'optional.minor_operating_system_version': 1, 'optional.major_subsystem_version': 5, 'optional.minor_subsystem_version': 1, 'optional.sizeof_code': 5120, 'optional.sizeof_headers': 1024, 'optional.sizeof_heap_commit': 4096}


In [84]:
import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/malware_dataset/PE_Header.csv")

In [85]:
df

,SHA256,Type,e_magic,e_cblp,e_cp,e_crlc,e_cparhdr,e_minalloc,e_maxalloc,e_ss,...,SizeOfImage,SizeOfHeaders,CheckSum,Subsystem,DllCharacteristics,SizeOfStackReserve,SizeOfHeapReserve,SizeOfHeapCommit,LoaderFlags,NumberOfRvaAndSizes
0,dacbe8cb72dd746539792a50e84965fefef73feaa07b5d...,0,23117,144,3,0,4,0,65535,0,...,139264,4096,0,2,1344,1048576,1048576,4096,0,16
1,d3dc7512ce75db33b2c3063fa99245e9ca9fe3b086462f...,0,23117,144,3,0,4,0,65535,0,...,49152,512,0,2,34112,1048576,1048576,4096,0,16
2,b350fac81533f02981dc2176ed17163177d92d9405758e...,0,23117,144,3,0,4,0,65535,0,...,532480,512,0,2,34144,1048576,1048576,4096,0,16
3,dfee618043a47b7b09305df0ca460559d9f567ee246c7b...,0,23117,144,3,0,4,0,65535,0,...,1368064,4096,1366781,2,1024,1048576,1048576,4096,0,16
4,c7b2e4e4fb2fcc44c953673ff57c3d14bdf5d2008f35e9...,0,23117,144,3,0,4,0,65535,0,...,32768,512,64362,2,1344,1048576,1048576,4096,0,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29802,d1c923ec32d09be96adf3ba85e1855402f782b74437534...,6,23117,144,3,0,4,0,65535,0,...,974848,512,0,2,34112,1048576,1048576,4096,0,16
29803,d1dfc3203d99c0d54351bebd876762f27c0073e3cfef9a...,6,23117,144,3,0,4,0,65535,0,...,434176,512,0,2,34112,1048576,1048576,4096,0,16
29804,d1fea52507fc97ff419f8dd2ea8ecf689fb7c066cf8f18...,6,23117,144,3,0,4,0,65535,0,...,958464,512,0,2,34112,1048576,1048576,4096,0,16
29805,d201d8488d316c9ab7d537545efeda284568a652f23db1...,6,23117,144,3,0,4,0,65535,0,...,1114112,512,0,2,34112,1048576,1048576,4096,0,16


In [86]:
df.columns

Index(['SHA256', 'Type', 'e_magic', 'e_cblp', 'e_cp', 'e_crlc', 'e_cparhdr',
       'e_minalloc', 'e_maxalloc', 'e_ss', 'e_sp', 'e_csum', 'e_ip', 'e_cs',
       'e_lfarlc', 'e_ovno', 'e_oemid', 'e_oeminfo', 'e_lfanew', 'Machine',
       'NumberOfSections', 'TimeDateStamp', 'PointerToSymbolTable',
       'NumberOfSymbols', 'SizeOfOptionalHeader', 'Characteristics', 'Magic',
       'MajorLinkerVersion', 'MinorLinkerVersion', 'SizeOfCode',
       'SizeOfInitializedData', 'SizeOfUninitializedData',
       'AddressOfEntryPoint', 'BaseOfCode', 'ImageBase', 'SectionAlignment',
       'FileAlignment', 'MajorOperatingSystemVersion',
       'MinorOperatingSystemVersion', 'MajorImageVersion', 'MinorImageVersion',
       'MajorSubsystemVersion', 'MinorSubsystemVersion', 'Reserved1',
       'SizeOfImage', 'SizeOfHeaders', 'CheckSum', 'Subsystem',
       'DllCharacteristics', 'SizeOfStackReserve', 'SizeOfHeapReserve',
       'SizeOfHeapCommit', 'LoaderFlags', 'NumberOfRvaAndSizes'],
      dtype='obj

In [87]:
# Load the dataset
df = pd.read_csv('/content/drive/MyDrive/malware_dataset/PE_Header.csv')

# Drop irrelevant columns and keep only PE header-related columns
relevant_columns = ['Type', 'e_magic', 'e_cblp', 'e_cp', 'e_crlc', 'e_cparhdr',
       'e_minalloc', 'e_maxalloc', 'e_ss', 'e_sp', 'e_csum', 'e_ip', 'e_cs',
       'e_lfarlc', 'e_ovno', 'e_oemid', 'e_oeminfo', 'e_lfanew', 'Machine',
       'NumberOfSections', 'TimeDateStamp', 'PointerToSymbolTable',
       'NumberOfSymbols', 'SizeOfOptionalHeader', 'Characteristics', 'Magic',
       'MajorLinkerVersion', 'MinorLinkerVersion', 'SizeOfCode',
       'SizeOfInitializedData', 'SizeOfUninitializedData',
       'AddressOfEntryPoint', 'BaseOfCode', 'ImageBase', 'SectionAlignment',
       'FileAlignment', 'MajorOperatingSystemVersion',
       'MinorOperatingSystemVersion', 'MajorImageVersion', 'MinorImageVersion',
       'MajorSubsystemVersion', 'MinorSubsystemVersion', 'Reserved1',
       'SizeOfImage', 'SizeOfHeaders', 'CheckSum', 'Subsystem',
       'DllCharacteristics', 'SizeOfStackReserve', 'SizeOfHeapReserve',
       'SizeOfHeapCommit', 'LoaderFlags', 'NumberOfRvaAndSizes']

df = df[relevant_columns]


In [88]:
df.dtypes

,0
Type,int64
e_magic,int64
e_cblp,int64
e_cp,int64
e_crlc,int64
e_cparhdr,int64
e_minalloc,int64
e_maxalloc,int64
e_ss,int64
e_sp,int64


In [89]:
from sklearn.model_selection import train_test_split

In [90]:
l1 = list(df.columns)



In [91]:
y = df['Type']
X = df[l1[1:]]


In [92]:
y.unique()
y.value_counts()

,count
Type,
4,5104
1,5047
3,4973
2,4864
5,4236
6,3706
0,1877


In [98]:
from imblearn.over_sampling import SMOTE
X_resampled, y_resampled = SMOTE().fit_resample(X,y)

In [94]:
y_resampled.value_counts()

,count
Type,
0,5104
1,5104
2,5104
3,5104
4,5104
5,5104
6,5104


In [99]:
X_train, X_test, y_train, y_test = train_test_split(X_resampled,y_resampled,test_size = 0.2, random_state= 42)

In [100]:
from sklearn.ensemble import RandomForestClassifier

In [103]:
model = RandomForestClassifier(n_estimators=250 , random_state=42)
model.fit(X_train, y_train)

RandomForestClassifier(n_estimators=250, random_state=42)

In [104]:
from sklearn.metrics import accuracy_score

y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 90.20%


In [107]:
import xgboost as xgb

In [111]:
model = xgb.XGBClassifier(random_state = 42)
model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [112]:
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 89.17%


In [116]:
import lightgbm as lgb

In [117]:
# Create a
model = lgb.LGBMClassifier(objective='multiclass', metric='multi_logloss')

# Train the model
model.fit(X_train, y_train)

# Make predictions on the testing set
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.015522 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4338
[LightGBM] [Info] Number of data points in the train set: 28582, number of used features: 50
[LightGBM] [Info] Start training from score -1.956780
[LightGBM] [Info] Start training from score -1.950363
[LightGBM] [Info] Start training from score -1.936196
[LightGBM] [Info] Start training from score -1.947906
[LightGBM] [Info] Start training from score -1.940815
[LightGBM] [Info] Start training from score -1.950117
[LightGBM] [Info] Start training from score -1.939354
Accuracy: 0.8897285194514414
